In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import zipfile
pd.set_option("display.max_columns", None)

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance

#Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix


In [ ]:

df = pd.read_csv('02-14-2018.csv')
df2 = pd .read_csv('02-15-2018.csv')
df3=pd.read_csv('02-16-2018.csv')
df4=pd.read_csv('02-21-2018.csv')

In [ ]:
data=pd.concat([df,df2,df3,df4],axis=0)
data.head ()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632086e+07,139.300036,56320958,56320761,112641719,5.632086e+07,139.300036,56320958,56320761,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632073e+07,114.551299,56320814,56320652,112641466,5.632073e+07,114.551299,56320814,56320652,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026634,5.631931e+07,301.934596,56319525,56319098,112638623,5.631931e+07,301.934596,56319525,56319098,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026634,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,82.600000,196.741237,976,0,227.300000,371.677892,544.161528,3.873587,2.689152e+05,247443.778966,673900,22,6453966,4.609976e+05,123109.423588,673900,229740,5637902,626433.555556,455082.214224,1167293,554,0,0,0,0,488,328,2.324152,1.549435,0,976,135.076923,277.834760,77192.153846,0,0,0,1,0,0,0,0,0,140.48,82.600000,227.300000,0,0,0,0,0,0,15,1239,10,2273,65535,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,81.642857,203.745545,976,0,200.818182,362.249864,380.733175,2.839597,3.668361e+05,511356.609733,1928102,21,8804066,6.772358e+05,532416.970959,1928102,246924,7715481,771548.100000,755543.082717,2174893,90,0,0,0,0,456,360,1.590174,1.249423,0,976,128.923077,279.763032,78267.353846,0,0,0,1,0,0,0,0,0,134.08,81.642857,200.818182,0,0,0,0,0,0,14,1143,11,2209,5808,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [ ]:
data.shape

(3459924, 80)

In [ ]:
data.describe()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.452726e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3459924.0,3459924.0,3459924.0,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3459924.0,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3459924.0,3459924.0,3459924.0,3459924.0,3459924.0,3459924.0,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06,3.459924e+06
mean,1.101415e+04,7.716380e+00,6.501136e+06,6.333459e+01,4.668518e+00,2.285236e+03,2.954761e+03,2.381847e+02,6.857648e+00,5.539445e+01,9.499782e+01,2.904582e+02,1.631194e+01,9.567794e+01,1.220856e+02,inf,inf,1.435481e+06,1.137804e+06,4.126691e+06,2.621121e+05,6.260188e+06,1.683603e+06,1.141502e+06,3.960455e+06,4.068539e+05,4.796573e+06,5.788646e+05,5.457000e+05,1.657317e+06,2.243362e+05,2.360399e-02,0.0,0.0,0.0,5.795050e+02,1.060725e+02,6.423315e+04,4.908908e+04,6.784791e+00,3.776119e+02,7.154578e+01,1.229573e+02,4.117454e+04,2.882433e-03,2.360399e-02,1.814381e-01,4.509556e-01,3.775950e-01,5.201010e-02,0.0,1.814372e-01,4.566031e-01,8.201821e+01,5.539445e+01,9.567794e+01,0.0,0.0,0.0,0.0,0.0,0.0,6.333459e+01,2.285236e+03,4.668518e+00,2.954761e+03,2.043511e+04,6.971669e+03,6.028396e+01,2.124712e+01,5.192638e+04,2.192218e+04,8.479359e+04,3.898802e+04,2.344429e+06,3.065953e+05,2.952450e+06,1.972388e+06
std,2.156571e+04,4.103284e+00,6.940808e+08,2.698592e+03,7.901685e+01,8.821539e+04,1.135664e+05,3.581070e+02,1.858410e+01,7.190635e+01,1.483505e+02,4.340419e+02,4.239676e+01,1.462945e+02,1.839836e+02,NaN,NaN,1.835667e+08,4.036986e+08,8.424113e+08,9.033040e+08,6.940758e+08,1.835968e+08,4.036965e+08,8.424091e+08,9.033107e+08,2.085158e+07,4.248146e+06,2.737775e+06,8.439549e+06,3.665599e+06,1.518119e-01,0.0,0.0,0.0,2.159834e+04,1.585467e+03,2.335354e+05,1.697565e+05,1.786581e+01,4.941563e+02,9.854535e+01,1.614188e+02,1.200692e+05,5.361087e-02,1.518119e-01,3.853808e-01,4.975889e-01,4.847856e-01,2.220474e-01,0.0,3.853801e-01,6.048406e-01,1.047736e+02,7.190635e+01,1.462945e+02,0.0,0.0,0.0,0.0,0.0,0.0,2.698592e+03,8.821539e+04,7.901685e+01,1.135664e+05,2.325229e+04,1.650189e+04,2.698465e+03,8.902633e+00,6.229245e+05,2.474900e+05,7.971514e+05,5.761744e+05,2.981991e+08,2.103014e+08,8.379863e+08,1.274919e+07
min,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e

In [ ]:


data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3145725 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Timestamp          object 
 3   Flow Duration      int64  
 4   Tot Fwd Pkts       int64  
 5   Tot Bwd Pkts       int64  
 6   TotLen Fwd Pkts    int64  
 7   TotLen Bwd Pkts    int64  
 8   Fwd Pkt Len Max    int64  
 9   Fwd Pkt Len Min    int64  
 10  Fwd Pkt Len Mean   float64
 11  Fwd Pkt Len Std    float64
 12  Bwd Pkt Len Max    int64  
 13  Bwd Pkt Len Min    int64  
 14  Bwd Pkt Len Mean   float64
 15  Bwd Pkt Len Std    float64
 16  Flow Byts/s        float64
 17  Flow Pkts/s        float64
 18  Flow IAT Mean      float64
 19  Flow IAT Std       float64
 20  Flow IAT Max       int64  
 21  Flow IAT Min       int64  
 22  Fwd IAT Tot        int64  
 23  Fwd IAT Mean       float64
 24  Fwd IAT Std        float64
 25  Fwd IAT Max       

In [ ]:
NUMBER = 10000000000

# Assigning large values instead of infinity
data["Flow Byts/s"].replace(np.inf,np.nan,inplace=True)
max(data["Flow Byts/s"])
data["Flow Pkts/s"].replace(np.inf,np.nan,inplace=True)
max(data["Flow Pkts/s"])

4000000.0

In [ ]:
pd.set_option('display.max_rows', 500)
data.isna().sum()

Dst Port                 0
Protocol                 0
Timestamp                0
Flow Duration            0
Tot Fwd Pkts             0
Tot Bwd Pkts             0
TotLen Fwd Pkts          0
TotLen Bwd Pkts          0
Fwd Pkt Len Max          0
Fwd Pkt Len Min          0
Fwd Pkt Len Mean         0
Fwd Pkt Len Std          0
Bwd Pkt Len Max          0
Bwd Pkt Len Min          0
Bwd Pkt Len Mean         0
Bwd Pkt Len Std          0
Flow Byts/s          11851
Flow Pkts/s          11851
Flow IAT Mean            0
Flow IAT Std             0
Flow IAT Max             0
Flow IAT Min             0
Fwd IAT Tot              0
Fwd IAT Mean             0
Fwd IAT Std              0
Fwd IAT Max              0
Fwd IAT Min              0
Bwd IAT Tot              0
Bwd IAT Mean             0
Bwd IAT Std              0
Bwd IAT Max              0
Bwd IAT Min              0
Fwd PSH Flags            0
Bwd PSH Flags            0
Fwd URG Flags            0
Bwd URG Flags            0
Fwd Header Len           0
B

In [ ]:
#assigning a large number instead of infinity
data["Flow Byts/s"].replace(np.nan,NUMBER,inplace=True)
data["Flow Pkts/s"].replace(np.nan,NUMBER,inplace=True)

In [ ]:
pd.set_option('display.max_rows', 500)
data.isna().sum()

Dst Port             0
Protocol             0
Timestamp            0
Flow Duration        0
Tot Fwd Pkts         0
Tot Bwd Pkts         0
TotLen Fwd Pkts      0
TotLen Bwd Pkts      0
Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Flow IAT Mean        0
Flow IAT Std         0
Flow IAT Max         0
Flow IAT Min         0
Fwd IAT Tot          0
Fwd IAT Mean         0
Fwd IAT Std          0
Fwd IAT Max          0
Fwd IAT Min          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd PSH Flags        0
Bwd PSH Flags        0
Fwd URG Flags        0
Bwd URG Flags        0
Fwd Header Len       0
Bwd Header Len       0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std

In [ ]:
data['Init Bwd Win Byts'].fillna(data['Init Bwd Win Byts'].mean(),inplace=True)
data['Init Fwd Win Byts'].fillna(data['Init Fwd Win Byts'].mean(),inplace=True) 

In [ ]:
pd.set_option('display.max_rows', 500)
data.isna().sum()

In [ ]:
data["Label"].value_counts()

Benign                      2124287
DDOS attack-HOIC             686012
FTP-BruteForce               193360
SSH-Bruteforce               187589
DoS attacks-Hulk             123013
DoS attacks-SlowHTTPTest      91434
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
DDOS attack-LOIC-UDP           1730
Name: Label, dtype: int64

In [ ]:
data['Label'].dropna(inplace=True,axis=0)
data=data.loc[(data["Label"].str.contains('DoS', na=False)) | (data["Label"]=='Benign') ]

In [ ]:
#data = df.dropna()
target = data["Label"]
data = data.drop(columns = "Label", axis = 1)


#data = data.loc[:, (df != 0).any(axis=0)]

target =pd.get_dummies(target)

cor_B = data.corrwith(other = target["Benign"],method = 'pearson')
#cor_F = data.corrwith(other = target["FTP-BruteForce"],method = 'pearson')

In [ ]:
data.drop('Timestamp',axis=1,inplace=True)

In [ ]:
data.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,112641719,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632086e+07,139.300036,56320958,56320761,112641719,5.632086e+07,139.300036,56320958,56320761,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761
1,0,0,112641466,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632073e+07,114.551299,56320814,56320652,112641466,5.632073e+07,114.551299,56320814,56320652,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652
2,0,0,112638623,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026634,5.631931e+07,301.934596,56319525,56319098,112638623,5.631931e+07,301.934596,56319525,56319098,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.026634,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098
3,22,6,6453966,15,10,1239,2273,744,0,82.600000,196.741237,976,0,227.300000,371.677892,544.161528,3.873587,2.689152e+05,247443.778966,673900,22,6453966,4.609976e+05,123109.423588,673900,229740,5637902,626433.555556,455082.214224,1167293,554,0,0,0,0,488,328,2.324152,1.549435,0,976,135.076923,277.834760,77192.153846,0,0,0,1,0,0,0,0,0,140.48,82.600000,227.300000,0,0,0,0,0,0,15,1239,10,2273,65535,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0
4,22,6,8804066,14,11,1143,2209,744,0,81.642857,203.745545,976,0,200.818182,362.249864,380.733175,2.839597,3.668361e+05,511356.609733,1928102,21,8804066,6.772358e+05,532416.970959,1928102,246924,7715481,771548.100000,755543.082717,2174893,90,0,0,0,0,456,360,1.590174,1.249423,0,976,128.923077,279.763032,78267.353846,0,0,0,1,0,0,0,0,0,134.08,81.642857,200.818182,0,0,0,0,0,0,14,1143,11,2209,5808,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0


In [ ]:
X=data
Y=target


In [ ]:
Y.columns

Index(['Benign', 'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest'], dtype='object')

In [ ]:
Y.drop(['DoS attacks-Hulk','DoS attacks-SlowHTTPTest'],inplace=True,axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold, chi2, SequentialFeatureSelector, SelectKBest
std=StandardScaler()
X_new=std.fit_transform(X)
encoder = LabelEncoder()
Y_new = encoder.fit_transform(Y)
k_best_chi = SelectKBest(chi2, k=50)
k_best_chi.fit_transform(abs(X_new), Y_new)
selected_features_chi = k_best_chi.get_feature_names_out(input_features=X.columns)
selected_features_chi

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
       'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Min', 'Fwd IAT Tot',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'ECE Flag Cnt',
       'Pkt Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts',
       'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
       'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min'],
      dtype=object)

Decision Tree Classifier

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_new , test_size=0.4, random_state=1) # 70% training and 30% test
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
acc_dt=accuracy_score(y_test, y_pred)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.9999895451404245


Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Random Forest Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
acc_rf=metrics.accuracy_score(y_test, y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9999874541685093


In [ ]:

from sklearn.compose import make_column_selector as selector
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [ ]:
#cleaning data
for i in numerical_columns:
  data[i].fillna(data[i].mean())


In [ ]:
target.columns

Index(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Hulk',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris'],
      dtype='object')

In [ ]:
target.drop(['DoS attacks-GoldenEye', 'DoS attacks-Hulk',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris'],inplace=True,axis=1)

In [ ]:
target.value_counts()

Benign
1         2124287
0          266945
dtype: int64

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

Logistic Regression

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
model = make_pipeline(preprocessor, LogisticRegression(max_iter = 500))

from sklearn import set_config
set_config(display='diagram') #interactive diagram
model



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  []),
                                                 ('standard-scaler',
                                                  StandardScaler(),
                                                  ['Dst Port', 'Protocol',
                                                   'Flow Duration',
                                                   'Tot Fwd Pkts',
                                                   'Tot Bwd Pkts',
                                                   'TotLen Fwd Pkts',
                                                   'TotLen Bwd Pkts',
                                                   'Fwd Pkt Len Max',
                                                   'Fwd Pkt Len Min',
                                                   'Fwd Pkt Len Mean',
                                                   'Fwd Pkt Len Std',
                                                   'Bwd Pkt Len Max',
                                                   'Bwd Pkt Len Min',
                                                   'Bwd Pkt Len Mean',
                                                   'Bwd Pkt Len Std',
                                                   'Flow Byts/s', 'Flow Pkts/s',
                                                   'Flow IAT Mean',
                                                   'Flow IAT Std',
                                                   'Flow IAT Max',
                                                   'Flow IAT Min',
                                                   'Fwd IAT Tot',
                                                   'Fwd IAT Mean',
                                                   'Fwd IAT Std', 'Fwd IAT Max',
                                                   'Fwd IAT Min', 'Bwd IAT Tot',
                                                   'Bwd IAT Mean',
                                                   'Bwd IAT Std', 'Bwd IAT Max', ...])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [ ]:
categorical_columns 


[]

In [ ]:
from sklearn.model_selection import cross_validate
#target = target.drop(["DoS attacks-GoldenEye",	"DoS attacks-Hulk","DoS attacks-SlowHTTPTest"], axis =1) #avoiding dummy variable trap
cv_results = cross_validate(model, data, target, cv=5)
cv_results


In [ ]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.9875 +/- 0.03
